# Singular - Crypto 485

Alice and Bob calculated a shared key on the elliptic curve
$y^2 = x^3 + 330762886318172394930696774593722907073441522749x^2 + 6688528763308432271990130594743714957884433976x + 759214505060964991648440027744756938681220132782$

$p = 785482254973602570424508065997142892171538672071$

$G = (1, 68596750097555148647236998220450053605331891340)$

(Alice's public key) $P = d1 * G = (453762742842106273626661098428675073042272925939, 680431771406393872682158079307720147623468587944)$

(Bob's poblic key) $Q = d2 * G = (353016783569351064519522488538358652176885848450, 287096710721721383077746502546881354857243084036)$

They have calculated $K = d1 * d2 * G$. They have taken $K$'s x coordinate in decimal and took sha256 of it and used it for AES ECB to encrypt the flag.

Here is the encrypted flag: 480fd106c9a637d22fddd814965742236eb314c1b8fb68e70a7c7445ff04476082f8b9026c49d27110ba41b95e9f51dc

In [1]:
from Crypto.Cipher import AES
from hashlib import sha256

In [2]:
# Task data
enc_flag = '480fd106c9a637d22fddd814965742236eb314c1b8fb68e70a7c7445ff04476082f8b9026c49d27110ba41b95e9f51dc'.decode('hex')
p = 785482254973602570424508065997142892171538672071
F = GF(p)
_.<x> = F[]

P = (F(453762742842106273626661098428675073042272925939), F(680431771406393872682158079307720147623468587944))
Q = (F(353016783569351064519522488538358652176885848450), F(287096710721721383077746502546881354857243084036))
G = (F(1), F(68596750097555148647236998220450053605331891340)) 

# Solution

TL;DR: Diffie-Hellman on an elliptic curve where computing discrete logarithms is easy. Recover the private keys and decrypt the flag.

ECDH(Elliptic Curve Diffie-Hellman) is a variant of the Diffie-Hellman key exchange that takes place over an elliptic curve rather than on a subset of the integers. Because computing discrete logarithms is normally hard on elliptic curve, recovering the participants' private keys from the messages they exchange is also hard. However there are certain special ("singular") curves on which computing discrete logarithms is easy, which completely breaks ECDH's security.

Because singular curves are not technically elliptic curves, Sage will throw an exception if we try to create one. As the title suggests, the curve used in this task is singular.

In [3]:
try:
    EllipticCurve(F, [0, 330762886318172394930696774593722907073441522749, 0, 6688528763308432271990130594743714957884433976, 759214505060964991648440027744756938681220132782])
except ArithmeticError as e:
    print e

invariants (0, 330762886318172394930696774593722907073441522749, 0, 6688528763308432271990130594743714957884433976, 759214505060964991648440027744756938681220132782) define a singular curve


This happens because the polynomial on the right hand side of the curve's equation is not irreducible. In our case it's of the form $(x - r)^3$.

In [4]:
poly = x^3 + 330762886318172394930696774593722907073441522749*x^2 + 6688528763308432271990130594743714957884433976*x + 759214505060964991648440027744756938681220132782
poly.factor()

(x + 372081713763924988451734946863621933081660064940)^3

It turns out that a curve of the form $y^2 = x^3$ is isomorphic to $(Z_p, +)$. What that means is that there is a one-to-one correspondence between the points of the curve and the integers between 0 and $p - 1$, and that adding two points on the curve is the same as adding the two corresponding integers modulo $p$. What that means is that we can compute the image of the two public keys in $Z_p$, then divide them by the image of $G$ to obtain the private keys.

For our curve the isomorphism is $(x, y) \leftrightarrow \frac{x}{y}$.

However because our curve is actually of the form $y^2 = (x - r)^3$ we first have to do a change of variable $x' = x - r$ (essentially shift the curve to the left).

In [5]:
# Map our points to points on the translated curve
r = poly.roots()[0][0]
P1 = (P[0] - r, P[1])
Q1 = (Q[0] - r, Q[1])
G1 = (G[0] - r, G[1])

# Then map them to integers
P2 = P1[0] / P1[1]
Q2 = Q1[0] / Q1[1]
G2 = G1[0] / G1[1]

Now recovering Alice's public key is as easy as doing a division modulo $p$.

In [6]:
d1 = P2 / G2

Finally we can compute the image of the shared secret and do the above process in reverse to map it to a point on the original curve, then use the result to decrypt the flag.

In [7]:
key1 = Q2 * d1

# Map to a point on the translated curve
key_x = (x^3 - (x / key1)^2).roots()[0][0]
Key2 = (key_x, key_x / key1)

# Map to a point on the original curve
Key3 = (Key2[0] + r, Key2[1])

In [8]:
# Decrypt the flag
aeskey = sha256(str(Key3[0])).digest()
cipher = AES.new(aeskey, mode=AES.MODE_ECB)
cipher.decrypt(enc_flag)

'hackim19{w0ah_math_i5_quite_fun_a57f8e21}\x07\x07\x07\x07\x07\x07\x07'